In [ ]:
%%capture
%pip install resonances pandas numpy matplotlib requests

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import resonances
from resonances.util import convert_mjd_to_date

import base64
import requests

In [ ]:
sim = resonances.Simulation()
sim.create_solar_system(date=convert_mjd_to_date(60200.0))
sim.plot = False
sim.save = True
resonances.config.set('plot.path', f"./cache")

r_bodies = ['463', '2348', '2487', '2640', '2791', '3048', '3293', '3699', '3716', '3733']
t_bodies = ['5230', '7325', '10774', '13944', '16701', '17925', '20849', '21519', '22947', '26121']
n_bodies = ['25', '192', '273', '572', '1011', '1066', '1273', '1504', '1551', '1660']

bodies = r_bodies + t_bodies + n_bodies

for body in bodies:
    sim.add_body(body, resonances.create_mmr('4J-2S-1'), name=body)
sim.dt = 1
sim.run()

In [ ]:
for body in sim.bodies:
    filename = f'cache/data-{body.index_in_simulation}-{body.name}.csv'
    df = pd.read_csv(filename)
    df.loc[df['angle'] > np.pi, 'angle'] -= 2 * np.pi

    plt.figure(figsize=(8, 2))
    plt.scatter(df['times'], df['angle'], color='black', marker='.', linewidths=0.5, s=1.0)
    plt.xlim(0, 100000)
    plt.ylim(-np.pi, np.pi)
    plt.tight_layout()
    plt.savefig(f'cache/{body.name}.png', format='png', bbox_inches='tight')
    plt.show()

In [ ]:
prompt = '''
I want you to act a scientist–astronomer. You will get an image uploaded. The image contains the plot of the resonant angle of an asteroid vs time (from 0 to 100000 years). The limits of OY axis are -pi and pi. The resonant angle cannot exceed these limits.

It is known that if the resonant angle librates, then the asteroid is trapped in the resonance. Librations mean oscillations, like sine. It means that the curve is within some limits (i.e., +2, or +1) and does not come close to the borders (-pi and pi). 

The opposite situation is when the resonant angle circulates. It means that the curve is not limited and can reach the borders of the plot. In our case, if the resonant angle is greater than pi or less than -pi, then we add or substract 2pi to the resonant angle to make it within the limits. Therefore, in the case of circulation, the pattern will be like linear curves parallel each other.

I want you to assess visually whether the resonant angle librates if you were a human looking at this image.

There are three possible cases:

1. The resonant angle librates all the time (from 0 to 100000). Then you should reply 'pure'.
2. The resonant angle could librate some significant time, but in other time is circulates. Let's assume that by significant I mean 20000 years. In this case, you should write 'transient'.
3. Otherwise, when the resonant angle circulates most of the time, please write 'non-resonant'.

As output, I want you only to print one word: pure, transient, or non-resonant. If you are not sure, write 'I do not know'. You will get tips if you perform the identification correctly.
'''

In [ ]:
api_key = "YOUR-OPEN-AI-API-KEY"

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

for body in sim.bodies:
  image_path = f"cache/{body.name}.png"
  base64_image = encode_image(image_path)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 500
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  data = response.json()
  # print(response.json())
  print(f"{body.name},{data.get('choices')[0].get('message').get('content')}")
